In [1]:
# Import modules
import numpy as np
import math
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier 

In [2]:
# Fungsi-fungsi aktivasi
def linear(x, derivative = False):
  if not (derivative):
    return x
  return np.ones_like(x)

def sigmoid(x, derivative = False):
  if not (derivative):
    return 1 / (1 + np.exp(-x))
  return sigmoid(x)*(1 - sigmoid(x))

def relu(x, derivative = False):
  if not (derivative):
    return np.maximum(0, x)
  return np.where(x >= 0, 1, 0)

def softmax(x, derivative = False):
    if not (derivative):
        ex = np.exp(x)
        return ex / np.sum(ex)
    result = np.zeros(x.shape)
    for i in range(x.shape[1]):
        temp = x[:,i].reshape(-1,1)
        resTemp = np.diagflat(temp) - np.dot(temp, temp.T)
        result[:,i] = np.sum(resTemp, axis=1)
    return result

activation_function = {
    "Linear": linear,
    "Sigmoid": sigmoid,
    "ReLU": relu,
    "Softmax": softmax,
}

In [3]:
# Fungsi loss

# Sum of squared errors (Linear, Sigmoid, ReLU)
def sum_of_squared_errors(target, output):
    return 0.5 * np.sum((target - output)**2)

# Softmax
def cross_entropy(target):
    result = (-1)*math.log(target)
    return result

cost_function = {
    "Linear": sum_of_squared_errors,
    "Sigmoid": sum_of_squared_errors,
    "ReLU": sum_of_squared_errors,
    "Softmax": cross_entropy,
}

In [4]:
class Layer():
  def __init__(self, weight, bias, activation, neurons):
    if activation not in ['Linear', 'Sigmoid', 'ReLU', 'Softmax']:
          raise NotImplementedError("Layer activation `%s` is not implemented." 
                                      % activation)
    self.weight = weight
    self.bias = bias
    self.activation = activation
    self.delta_weight = np.zeros(weight.shape)
    self.delta_bias = np.zeros(bias.shape)

  def net(self):
    input_T = input.T
    result = np.dot(self.weight, input_T) + self.bias
    return result
    
  def forward_propagation(self, input):
    result = net = self.net()
    return activation_function[self.activation](result)

  def calculate_error(self, target, output):
    cost_function[self.activation](target) if self.activation == "Softmax" else cost_function[self.activation](target, output)
        
class NeuralNetwork():
  def __init__(self, learning_rate, error_threshold, max_iter, batch_size):
    self.layers = []
    self.learning_rate = learning_rate
    self.error_threshold = error_threshold 
    self.max_iter = max_iter
    self.batch_size = batch_size
  
  def summary(self):
    print("Jumlah layer: ", len(self.layers))
    for i, layer in enumerate(self.layers):
      print("============================================================")
      print('Layer {} (Activation: "{}", Units: {})'.format(i+1, layer.activation, len(layer.weight)))
      print("Weight:")
      print(np.array(layer.weight))
      print("Bias:")
      print(np.array(layer.bias))
    print("============================================================")

  def add(self, layer):
    self.layers.append(layer)
  
  def predict(self, input):
    arr = np.array(input)
    if arr.ndim == 1:
      instance = arr
      instance_res = instance
      for layer in self.layers:
        instance_res = layer.forward_propagation(instance_res)
      return instance_res.tolist()

    batch = arr
    batch_res = []
    for instance in batch:
      instance_res = instance
      for layer in self.layers:
        instance_res = layer.forward_propagation(instance_res)
      
      batch_res.append(instance_res.tolist())
    
    return batch_res
        
  def load_file(self, filename):
    '''
    File format
    <depth>
    <units> <activation function>
    <weight0> 
    <bias0>
    '''
    with open(filename, 'r') as file:
      depth = int(file.readline().strip())
      for i in range (depth):
        line = file.readline().strip().split()
        unit = int(line[0])
        activation = line[1]

        # Weight Matrix
        weight = []
        for j in range(unit):
          weight.append(list(map(float, file.readline().strip().split())))

        # Bias Matrix
        bias = list(map(float, file.readline().strip().split()))
        
        # Add layer
        layer = Layer(weight, bias, activation)
        self.add(layer)
      
      # End of file
    # Close file
    print('File loaded. Model detected')

  def shuffle(X, y):
    data = np.stack((X, y), axis=1)
    np.random.shuffle(data)

    return data

  def create_batch(self, X, y):
    batch_x = []
    batch_y = []
    epoch = math.ceil(len(X) / self.batch_size)

    # Shuffle data
    data  = self.shuffle(X, y)

    for i in range(0, epoch):
      head = i * self.batch_size
      tail = (i + 1) * self.batch_size
      mini_batch = data[head : tail]
      batch_x.append(data[:,0])
      batch_y.append(data[:,1])

    return batch_x, batch_y

  def delta(self, target, output):
    return output * (1 - output) * (target - output)

  def backward_propagation(self, X, y, output):
    gradient = 1
    for i, layer in reversed(list(enumerate(self.layers))):
      gradient *= layer.calculate_error(target = y, output = output, derivative = True)
      if (i == len(self.layers)-1):
        # gradient *= layer.calculate_error(target = y, output = output, derivative = True)
        if (layer.activation != "Softmax"):
          #compute gradient -> calculate derivative error of w
          gradient = (-1) * self.delta(target = y, output = output) * X
        
        #softmax
        else:
          #compute gradient -> calculate derivative error of w
          gradient = softmax(gradient, derivative=True) * X

      #hidden layer  
      else:
        if (layer.activation != "Softmax"):
          pass
          # gradient *= (-1) * output * (1 - output) * np.sum()
        else:
          pass
          
    return gradient

  def mgd(self, X, y):
    for iteration in range(0, self.max_iter):
      # Divide into batch
      batch_x, batch_y = self.create_batch(X, y)
      batches = len(batch_x)
      
      error = 0  
      for batch in range(0, batches):
        X_train = batch_x[batch]
        y_train = batch_y[batch]

        # Forward propagation on input
        prediction = self.predict(X_train)

        # Compute cost
        error += self.layers[-1].calculate_error(y_train, prediction)
        
        # Backward propagation to count gradient
        gradient = self.backward_propagation(X_train, y_train, prediction)

        # Update dw
        for layer in self.layers:
          layer.delta_weight = -1 * self.learning_rate * gradient

        # Update weight
        for layer in self.layers:
          layer.weight += layer.delta_weight
          layer.bias += layer.delta_bias
          
          # Reset delta value
          layer.delta_weight = np.zeros(layer.weight.shape)
          layer.delta_bias = np.zeros(layer.bias.shape)
      
      error *= 1 / len(X)
      if error <= self.error_threshold:
        print("Error is lower or equal than error threshold")
        print("Ended in {} iterations".format(iteration))
        return
    
    print("Reached maximum iterations")
    print("Ended in {} iterations".format(self.max_iter))
    return